In [1]:
#Importing dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pymongo
import datetime
from flask import Flask
from sqlalchemy import create_engine
import json
#Avoiding setting with copy warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing Redfin Housing Market State Data
raw_state_df = pd.read_csv("Resources/state_market_tracker.tsv000", sep='\t')
raw_state_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2019-10-01,2019-10-31,30,state,4,23,f,Oklahoma,NaN,Oklahoma,...,0.098560,0.209877,0.083561,0.053355,0.301282,-0.240768,-0.122115,South Region,NaN,2022-01-09 14:29:56
1,2021-07-01,2021-07-31,30,state,4,40,f,Vermont,NaN,Vermont,...,0.251473,0.133696,0.016402,-0.000131,0.424404,-0.064422,0.076156,Northeast Region,NaN,2022-01-09 14:29:56
2,2016-08-01,2016-08-31,30,state,4,10,f,New Hampshire,NaN,New Hampshire,...,0.089756,0.149826,0.009592,0.002687,0.206101,0.061385,0.016779,Northeast Region,NaN,2022-01-09 14:29:56
3,2013-04-01,2013-04-30,30,state,4,51,f,Mississippi,NaN,Mississippi,...,-0.008566,NaN,NaN,NaN,0.044777,-0.014869,-0.005006,South Region,NaN,2022-01-09 14:29:56
4,2019-12-01,2019-12-31,30,state,4,2,f,Missouri,NaN,Missouri,...,-0.017031,0.205339,-0.123381,-0.000410,0.252082,-0.116423,0.095007,Midwest Region,NaN,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27073,2012-03-01,2012-03-31,30,state,4,6,f,Nebraska,NaN,Nebraska,...,0.021426,NaN,NaN,NaN,0.422780,0.042801,0.030384,Midwest Region,NaN,2022-01-09 14:29:56
27074,2016-07-01,2016-07-31,30,state,4,27,f,Idaho,NaN,Idaho,...,0.182390,NaN,NaN,NaN,0.534900,-0.117877,-0.004820,West Region,NaN,2022-01-09 14:29:56
27075,2020-11-01,2020-11-30,30,state,4,42,f,Virginia,NaN,Virginia,...,0.218939,0.126582,-0.048418,-0.006751,0.380567,-0.152381,0.194669,South Region,NaN,2022-01-09 14:29:56
27076,2021-08-01,2021-08-31,30,state,4,40,f,Vermont,NaN,Vermont,...,0.234272,0.158853,0.018284,0.022594,0.381659,-0.018233,0.098021,Northeast Region,NaN,2022-01-09 14:29:56


In [45]:
#Printing out column values to figure out what data we are interested in
print(raw_state_df.columns)

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',
 

In [3]:
#Checking for outliers
raw_state_df['property_type'].value_counts()

All Residential              5632
Single Family Residential    5632
Condo/Co-op                  5496
Multi-Family (2-4 Unit)      5406
Townhouse                    4912
Name: property_type, dtype: int64

In [4]:
#Checking DC for outliers
dc_df = raw_state_df[raw_state_df['state_code'] == 'DC']
dc_df['property_type'].value_counts()

Single Family Residential    120
Multi-Family (2-4 Unit)      120
Townhouse                    120
All Residential              120
Condo/Co-op                  120
Name: property_type, dtype: int64

In [5]:
#Dropping unnecessary columns by calling only the ones we need
state_df = raw_state_df[['state_code', 'period_end', 'median_sale_price', 
                         'median_ppsf', 'property_type']]

state_df.head()

,state_code,period_end,median_sale_price,median_ppsf,property_type
0,OK,2019-10-31,162200,77.0,Multi-Family (2-4 Unit)
1,VT,2021-07-31,317900,177.0,All Residential
2,NH,2016-08-31,200100,155.0,Condo/Co-op
3,MS,2013-04-30,129500,69.0,All Residential
4,MO,2019-12-31,152000,135.0,Condo/Co-op


In [6]:
#Viewing column types
state_df.dtypes

state_code            object
period_end            object
median_sale_price      int64
median_ppsf          float64
property_type         object
dtype: object

In [7]:
#Converting columns to strings and date/time
state_df['period_end'] = pd.to_datetime(state_df['period_end'], format='%Y-%m-%d')
state_df['state_code'] = state_df['state_code'].astype('string')
state_df['property_type'] = state_df['property_type'].astype('string')

state_df.dtypes

state_code                   string
period_end           datetime64[ns]
median_sale_price             int64
median_ppsf                 float64
property_type                string
dtype: object

In [8]:
#Droppihg years 2018 and older
state_df = state_df[state_df.period_end.dt.year > 2018]

In [9]:
state_df.rename(columns={'state_code':'state'},inplace=True)

In [46]:
#grab median sale price by state
state_df_mean = state_df.groupby(['state']).mean()
state_df_mean.sort_values(by='median_sale_price').head()

,median_sale_price,median_ppsf
state,,
OK,162056.111111,96.811111
OH,169773.888889,103.511111
MS,173332.022472,104.073034
IN,179582.876712,101.000000
MO,181113.333333,114.316667


In [60]:
#Getting the minimum price of a housing by state
min_state_df = state_df.groupby(['state']).min()
min_state_df

,period_end,median_sale_price,median_ppsf,property_type
state,,,,
AK,2019-01-31,160000,124.0,All Residential
AL,2019-01-31,30000,16.0,All Residential
AR,2019-01-31,110500,62.0,All Residential
AZ,2019-01-31,166000,37.0,All Residential
CA,2019-01-31,440900,323.0,All Residential
CO,2019-01-31,269000,132.0,All Residential
CT,2019-01-31,160200,70.0,All Residential
DC,2019-01-31,425000,214.0,All Residential
DE,2019-01-31,71200,36.0,All Residential


In [58]:
#Retreiving the lowest priced state out of the lowest house prices
min_price = min_state_df[min_state_df.median_sale_price==min_state_df.median_sale_price.min()]
min_price

,period_end,median_sale_price,median_ppsf,property_type
state,,,,
AL,2019-01-31,30000,16.0,All Residential


In [59]:
#Getting the maximum price of a house by state
max_state_df = state_df.groupby(['state']).max()
max_state_df

,period_end,median_sale_price,median_ppsf,property_type
state,,,,
AK,2021-12-31,452800,200.0,Single Family Residential
AL,2021-12-31,286000,195.0,Townhouse
AR,2021-12-31,271500,160.0,Townhouse
AZ,2021-12-31,476400,378.0,Townhouse
CA,2021-12-31,981500,559.0,Townhouse
CO,2021-12-31,761300,337.0,Townhouse
CT,2021-12-31,1750000,600.0,Townhouse
DC,2021-12-31,1275000,613.0,Townhouse
DE,2021-12-31,344800,171.0,Townhouse


In [61]:
#Retreiving the most expensive state out of the most expensive house prices
max_price = max_state_df[max_state_df.median_sale_price==max_state_df.median_sale_price.max()]
max_price

,period_end,median_sale_price,median_ppsf,property_type
state,,,,
HI,2021-12-31,1837000,1197.0,Townhouse


In [11]:
#grab median sale price by state
state_df_median = state_df.groupby(['state']).median()
state_df_median_sale = state_df_median.sort_values(by='median_sale_price')
state_df_median_sale.head()

,median_sale_price,median_ppsf
state,,
OK,171750.0,97.5
OH,173250.0,110.0
MO,179300.0,119.0
MS,179500.0,105.0
KY,183650.0,114.0


In [12]:
#grab median price per square foot by state
state_df_median_ppsf = state_df_median.sort_values(by='median_ppsf')
state_df_median_ppsf.head()

,median_sale_price,median_ppsf
state,,
OK,171750.0,97.5
IN,190350.0,102.0
MS,179500.0,105.0
AR,184000.0,106.0
WV,206900.0,109.0


In [13]:
#Importing Data Scientist Salaries info
raw_salary_df = pd.read_csv("Resources/raw_salary_data_scientist.csv")
raw_salary_df

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,tensor,hadoop,tableau,bi,flink,mongo,google_an,job_title_sim,seniority_by_title,Degree
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 - 1000,1973,...,0,0,1,1,0,0,0,data scientist,na,M
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+,1984,...,0,0,0,0,0,0,0,data scientist,na,M
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 - 1000,2010,...,0,0,0,0,0,0,0,data scientist,na,M
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 - 5000,1965,...,0,0,0,0,0,0,0,data scientist,na,na
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 - 200,1998,...,0,0,0,0,0,0,0,data scientist,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,950,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+,1830,...,0,0,0,0,0,0,0,other scientist,sr,M
738,951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 - 5000,2006,...,0,1,0,0,0,0,0,data engineer,sr,na
739,952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 - 1000,1984,...,0,0,0,0,0,0,0,other scientist,na,P
740,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 - 50,-1,...,0,0,0,0,0,0,0,data scientist,na,na


In [14]:
#Determining total number of states included in the original data
raw_salary_df['Job Location'].nunique()

37

In [15]:
#Dropping unneccesary columns by choosing only the ones we need
salary_df = raw_salary_df[['Job Location', 'Rating', 'Lower Salary', 'Upper Salary', 'Avg Salary(K)',  
                           'job_title_sim', 'seniority_by_title']]
salary_df

,Job Location,Rating,Lower Salary,Upper Salary,Avg Salary(K),job_title_sim,seniority_by_title
0,NM,3.8,53,91,72.0,data scientist,na
1,MD,3.4,63,112,87.5,data scientist,na
2,FL,4.8,80,90,85.0,data scientist,na
3,WA,3.8,56,97,76.5,data scientist,na
4,NY,2.9,86,143,114.5,data scientist,na
...,...,...,...,...,...,...,...
737,MA,3.9,58,111,84.5,other scientist,sr
738,TN,4.4,72,133,102.5,data engineer,sr
739,PA,2.6,56,91,73.5,other scientist,na
740,PA,3.2,95,160,127.5,data scientist,na


In [16]:
#Renaming location column to match other dataframe state column name
salary_df.rename(columns={"Job Location":"state", "Lower Salary":"lower_salary", "Upper Salary":"upper_salary",
                          "Avg Salary(K)":"avg_salary", "Rating":"rating"}, inplace=True)
salary_df.head()

,state,rating,lower_salary,upper_salary,avg_salary,job_title_sim,seniority_by_title
0,NM,3.8,53,91,72.0,data scientist,na
1,MD,3.4,63,112,87.5,data scientist,na
2,FL,4.8,80,90,85.0,data scientist,na
3,WA,3.8,56,97,76.5,data scientist,na
4,NY,2.9,86,143,114.5,data scientist,na


In [17]:
#Getting column types to see if we need to convert anything
salary_df.dtypes

state                  object
rating                float64
lower_salary            int64
upper_salary            int64
avg_salary            float64
job_title_sim          object
seniority_by_title     object
dtype: object

In [18]:
#convery types to strings
salary_df['state'] = salary_df['state'].astype('string')
salary_df['job_title_sim'] = salary_df['job_title_sim'].astype('string')
salary_df['seniority_by_title'] = salary_df['seniority_by_title'].astype('string')
salary_df.dtypes

state                  string
rating                float64
lower_salary            int64
upper_salary            int64
avg_salary            float64
job_title_sim          string
seniority_by_title     string
dtype: object

In [19]:
#determine values in seniority by title
salary_df['seniority_by_title'].value_counts()

na    519
sr    220
jr      3
Name: seniority_by_title, dtype: Int64

In [20]:
#drop senior and junior positions
salary_df = salary_df[salary_df.seniority_by_title != "sr"]
salary_df = salary_df[salary_df.seniority_by_title != "jr"]
salary_df['seniority_by_title'].value_counts()

na    519
Name: seniority_by_title, dtype: Int64

In [21]:
#number of states after data frame has been manipulated
salary_df['state'].nunique()

36

In [22]:
#grab median salary by state
salary_df_median = salary_df.groupby(['state']).median()
salary_df_median_sale = salary_df_median.sort_values(by='avg_salary', ascending=False)
salary_df_median_sale.head()

,rating,lower_salary,upper_salary,avg_salary
state,,,,
CA,3.6,80.0,140.5,110.25
NJ,3.6,85.0,134.0,109.50
KY,3.1,68.0,139.0,103.50
NC,3.7,65.0,126.0,99.50
OR,3.5,74.5,121.5,98.00


In [23]:
#Grabbing avg salary by state
salary_df_mean = salary_df.groupby(['state']).mean()
salary_df_mean_sale = salary_df_mean.sort_values(by='avg_salary', ascending=False)
salary_df_mean_sale.head()

,rating,lower_salary,upper_salary,avg_salary
state,,,,
IL,3.566667,84.333333,138.814815,111.574074
CA,3.606731,80.625000,139.490385,110.057692
NJ,3.691667,77.666667,131.083333,104.375000
DC,3.777778,82.000000,123.444444,102.722222
KY,3.500000,66.333333,127.666667,97.000000


In [24]:
#merge median dfs
merged_median_df = pd.merge(state_df_median, salary_df_median_sale, how = 'inner', on = 'state')
merged_median_df.head()

,median_sale_price,median_ppsf,rating,lower_salary,upper_salary,avg_salary
state,,,,,,
AL,192000.0,111.0,4.25,42.0,76.0,60.75
AZ,292350.0,185.5,3.45,57.0,84.0,69.00
CA,637500.0,417.5,3.60,80.0,140.5,110.25
CO,447000.0,192.0,4.00,64.0,111.0,87.50
CT,278800.0,155.0,3.00,54.0,71.0,62.50


In [25]:
#determining differential between median salary and price
merged_median_df['cost_vs_salary'] = merged_median_df['median_sale_price'] - (merged_median_df['avg_salary'] * 1000)
merged_median_df.sort_values(by='cost_vs_salary')

,median_sale_price,median_ppsf,rating,lower_salary,upper_salary,avg_salary,cost_vs_salary
state,,,,,,,
KY,183650.0,114.0,3.10,68.0,139.0,103.50,80150.0
OH,173250.0,110.0,3.50,56.0,117.0,86.50,86750.0
MI,189850.0,127.0,3.95,65.5,118.0,91.75,98100.0
MO,179300.0,119.0,4.10,44.0,101.0,70.50,108800.0
IN,190350.0,102.0,3.95,56.0,91.0,73.50,116850.0
AL,192000.0,111.0,4.25,42.0,76.0,60.75,131250.0
PA,221250.0,142.5,3.20,61.0,109.0,85.00,136250.0
IL,227500.0,147.5,3.70,64.0,105.0,87.50,140000.0
KS,234150.0,125.0,3.70,61.0,113.0,87.00,147150.0


In [26]:
#determining highest rated state for data science by median rating
merged_median_df.sort_values(by='rating', ascending=False)

,median_sale_price,median_ppsf,rating,lower_salary,upper_salary,avg_salary,cost_vs_salary
state,,,,,,,
GA,255800.0,141.5,4.70,60.0,99.0,79.50,176300.0
IA,200600.0,142.0,4.60,31.0,55.0,43.00,157600.0
MN,268750.0,151.5,4.45,60.0,111.0,85.50,183250.0
SC,250700.0,144.5,4.40,39.0,82.0,60.50,190200.0
AL,192000.0,111.0,4.25,42.0,76.0,60.75,131250.0
UT,374700.0,182.0,4.20,53.0,91.0,72.00,302700.0
MO,179300.0,119.0,4.10,44.0,101.0,70.50,108800.0
ID,345250.0,200.0,4.10,39.5,73.0,56.25,289000.0
VA,355350.0,196.0,4.00,65.0,113.0,88.00,267350.0


In [27]:
#determining highest rated state for data science by median rating
merged_median_df.sort_values(by='rating', ascending=False)

,median_sale_price,median_ppsf,rating,lower_salary,upper_salary,avg_salary,cost_vs_salary
state,,,,,,,
GA,255800.0,141.5,4.70,60.0,99.0,79.50,176300.0
IA,200600.0,142.0,4.60,31.0,55.0,43.00,157600.0
MN,268750.0,151.5,4.45,60.0,111.0,85.50,183250.0
SC,250700.0,144.5,4.40,39.0,82.0,60.50,190200.0
AL,192000.0,111.0,4.25,42.0,76.0,60.75,131250.0
UT,374700.0,182.0,4.20,53.0,91.0,72.00,302700.0
MO,179300.0,119.0,4.10,44.0,101.0,70.50,108800.0
ID,345250.0,200.0,4.10,39.5,73.0,56.25,289000.0
VA,355350.0,196.0,4.00,65.0,113.0,88.00,267350.0


In [28]:
#merge mean dfs
merged_mean_df = pd.merge(state_df_mean, salary_df_mean_sale, how = 'inner', on = 'state')
merged_mean_df.head()

,median_sale_price,median_ppsf,rating,lower_salary,upper_salary,avg_salary
state,,,,,,
AL,189148.888889,109.494382,4.212500,40.875000,75.000000,57.937500
AZ,294833.333333,197.627778,3.516667,62.166667,109.833333,86.000000
CA,670961.111111,420.272222,3.606731,80.625000,139.490385,110.057692
CO,458560.000000,207.455556,3.833333,58.333333,99.333333,78.833333
CT,359550.299401,184.581818,3.400000,57.600000,84.600000,71.100000


In [29]:
#determining differential between mean salary and price
merged_mean_df['cost_vs_salary'] = merged_mean_df['median_sale_price'] - (merged_mean_df['avg_salary'] * 1000)
merged_mean_df.sort_values(by='cost_vs_salary')

,median_sale_price,median_ppsf,rating,lower_salary,upper_salary,avg_salary,cost_vs_salary
state,,,,,,,
KY,186930.555556,111.733333,3.500000,66.333333,127.666667,97.000000,89930.555556
OH,169773.888889,103.511111,3.622222,53.222222,100.333333,76.777778,92996.111111
MO,181113.333333,114.316667,4.111111,59.666667,109.777778,84.722222,96391.111111
MI,191853.797468,116.960784,3.950000,65.500000,118.000000,91.750000,100103.797468
IN,179582.876712,101.000000,3.916667,52.333333,90.666667,71.500000,108082.876712
IL,232036.666667,147.055556,3.566667,84.333333,138.814815,111.574074,120462.592593
AL,189148.888889,109.494382,4.212500,40.875000,75.000000,57.937500,131211.388889
PA,220665.555556,146.250000,3.184211,64.947368,113.526316,89.236842,131428.713450
WI,206947.777778,126.877778,3.255556,52.000000,88.555556,70.277778,136670.000000


In [30]:
#determining highest rated state for data science by mean rating
merged_mean_df.sort_values(by='rating', ascending=False)

,median_sale_price,median_ppsf,rating,lower_salary,upper_salary,avg_salary,cost_vs_salary
state,,,,,,,
MN,266106.111111,158.616667,4.450000,60.000000,111.000000,85.500000,180606.111111
GA,264038.333333,150.194444,4.433333,61.333333,101.333333,81.333333,182705.000000
SC,260442.777778,149.544444,4.400000,39.000000,82.000000,60.500000,199942.777778
IA,202222.448980,138.301370,4.366667,38.666667,70.333333,54.500000,147722.448980
AL,189148.888889,109.494382,4.212500,40.875000,75.000000,57.937500,131211.388889
MO,181113.333333,114.316667,4.111111,59.666667,109.777778,84.722222,96391.111111
ID,356761.666667,213.622222,4.100000,39.500000,73.000000,56.250000,300511.666667
UT,392304.444444,203.638889,4.042857,51.857143,96.714286,74.285714,318018.730159
FL,276862.777778,174.522222,3.966667,57.666667,88.333333,73.000000,203862.777778


In [42]:
#Sorting dataframes by alphabetical order on state
state_df.sort_values("state", inplace=True)
salary_df.sort_values("state", inplace=True)
merged_mean_df.sort_values("state", inplace=True)
merged_median_df.sort_values("state", inplace=True)

In [33]:
# Initializing PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [34]:
#Defining the data science salary and housing data base in Mongo
db = client.salary_housing_db

In [35]:
#Making the main collections, data science salary and state housing data individually
state_housing = db.state_housing
ds_salary = db.ds_salary

In [36]:
#Inserting state housing dataframe into collection
state_housing.insert_many(state_df.to_dict('records'))

In [37]:
ds_salary.insert_many(salary_df.to_dict('records'))

In [38]:
#Declaring the median and mean collections
median = db.median
mean = db.mean

In [39]:
#Inserting median data frame in to median collection
median.insert_many(merged_median_df.to_dict('records'))

In [40]:
#Inserting mean data frame in to mean collection
mean.insert_many(merged_mean_df.to_dict('records'))